In [2]:
#@title Instalaçao de Bibliotecas

!pip install pymupdf
!pip install langchain
!pip install -U langchain-community
!pip install pypdf
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 48.5 MB/s eta 0:00:00


In [16]:
#@title Extração de Normas

import pdfplumber
import re
import pandas as pd

def extrair_texto_pdf(pdf_path):
    texto_completo = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texto_pagina = page.extract_text()
            if texto_pagina:
                texto_completo += texto_pagina + '\n'
    return texto_completo

def extrair_normas_legais(texto):
    # Padrões de normas legais mais comuns
    padroes = [
        # Padrões PGFN
        r'Parecer\s+SEI\s+(?:Nº\s+)?[\d\.]+/\d{4}/(?:CRJ/PGACET/)?PGFN(?:-MF|-ME)?',
        r'Nota\s+SEI\s+(?:nº\s+)?[\d\.]+/\d{4}/(?:CRJ/PGACET/)?PGFN(?:-MF|-ME)?',
        r'Parecer\s+PGFN/(?:CAT|PGA|CRJ)/(?:Nº\s+)?[\d\.]+(?:/\d{4})?',
        r'Nota\s+PGFN\s+(?:CRJ\s+)?(?:nº\s+)?[\d\.]+/\d{4}',
        r'(?:Ato\s+Declaratório\s+)?PGFN(?:/CRJ)?(?:/Nº\s+)?[\d\.]+(?:/\d{4})?',
        r'AD\s+PGFN\s+(?:nº\s+)?[\d\.]+',
        r'Ato\s+Declaratório\s+PGFN\s+(?:nº\s+)?[\d\.]+',

        # Instrução Normativa
        r'Instrução\s+Normativa\s+(?:RFB\s+)?(?:SRF\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Lei
        r'Lei\s+(?:Complementar\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Decreto
        r'Decreto\s+n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Ato Declaratório RFB
        r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:RFB\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Ato Declaratório SRF
        r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:SRF\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Ato Declaratório CST
        r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:CST\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Ato Declaratório COSIT
        r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:Cosit\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Ato Declaratório COSAR
        r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:Cosar\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Portaria
        r'Portaria\s+(?:RFB\s+)?(?:MF\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Resolução
        r'Resolução\s+(?:RFB\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Medida Provisória
        r'Medida\s+Provisória\s+n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Parecer Normativo
        r'Parecer\s+Normativo\s+(?:RFB\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # Solução de Consulta
        r'Solução\s+de\s+Consulta\s+(?:Cosit\s+)?n[º°]\s*[\d\.]+(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
        # RIR
        r'RIR/\d{4}'
    ]

    # Combinar todos os padrões em uma única expressão regular
    padrao_combinado = '|'.join(f'({padrao})' for padrao in padroes)

    # Encontrar todas as normas no texto
    normas = re.finditer(padrao_combinado, texto, re.IGNORECASE)

    # Processar e limpar as normas encontradas
    normas_encontradas = []
    for norma in normas:
        norma_texto = norma.group(0).strip()
        # Remover artigos e incisos
        norma_texto = re.sub(r',\s*art\.[^,]*', '', norma_texto, flags=re.IGNORECASE)
        norma_texto = re.sub(r',\s*inciso[^,]*', '', norma_texto, flags=re.IGNORECASE)
        if norma_texto and norma_texto not in normas_encontradas:
            normas_encontradas.append(norma_texto)

    return normas_encontradas

def extrair_normas(pdf_path):
    # Extrair texto do PDF
    texto_completo = extrair_texto_pdf(pdf_path)
    texto_completo = re.sub(r'\s+', ' ', texto_completo).strip()

    # Regex para identificar perguntas
    regex_pergunta = re.compile(r'(\d{1,3})\s*—\s*(.+?\?)')
    matches = list(regex_pergunta.finditer(texto_completo))

    # Coletar perguntas e normas
    normas_extracao = []

    for idx, match in enumerate(matches):
        numero_pergunta = match.group(1)
        texto_pergunta = match.group(2).strip()

        # Pegar texto de resposta
        end_pos = match.end()
        if idx + 1 < len(matches):
            start_next = matches[idx + 1].start()
            texto_resposta = texto_completo[end_pos:start_next].strip()
        else:
            texto_resposta = texto_completo[end_pos:].strip()

        # Extrair normas da resposta
        normas_encontradas = extrair_normas_legais(texto_resposta)

        normas_extracao.append({
            "Pergunta": f"{numero_pergunta} — {texto_pergunta}",
            "Norma": "; ".join(normas_encontradas) if normas_encontradas else "não tem norma legal para essa pergunta"
        })

    return normas_extracao

# Uso do código
pdf_path = "/content/P&R IRPF 2024 - v1.0 - 2024.05.03 (2).pdf"

# Extrair normas e converter para DataFrame
normas_extracao = extrair_normas(pdf_path)
df_normas = pd.DataFrame(normas_extracao)

# Exibir e salvar em Excel
print(df_normas.head(10))
df_normas.to_excel('normas_extraidas.xlsx', index=False)

# Baixar o arquivo no Colab
try:
    from google.colab import files
    files.download('normas_extraidas.xlsx')
except:
    print("Arquivo salvo localmente como 'normas_extraidas.xlsx'")

                                            Pergunta  \
0  001 — Quem está obrigado a apresentar a Declar...   
1  002 — Pessoa física desobrigada pode apresenta...   
2  003 — Contribuinte que é titular ou sócio de e...   
3  004 — Contribuinte, que participou de quadro s...   
4  005 — Contribuinte que constou como responsáve...   
5  006 — Existe limite de idade para a obrigatori...   
6  007 — Para verificação da obrigatoriedade de a...   
7  008 — A posse ou a propriedade de bens e direi...   
8  009 — O contribuinte deve apresentar uma Decla...   
9  010 — Dependente que possui depósito em conta ...   

                                               Norma  
0  Lei nº 11.196, de 21 de novembro de 2005; Lei ...  
1  Instrução Normativa RFB nº 2.178, de 5 de març...  
2  Instrução Normativa RFB nº 2.178, de 5 de març...  
3  Instrução Normativa RFB nº 2.178, de 5 de març...  
4  Instrução Normativa RFB nº 2.178, de 5 de març...  
5  RIR/2018; Decreto nº 9.580, de 22 de novembro ... 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
#@title Elminação de Duplicatas

import pandas as pd

def limpar_norma(norma):
    """Função auxiliar para limpar cada norma individual"""
    return norma.strip().replace('\n', ' ').replace('  ', ' ')

def analisar_normas_excel(caminho_arquivo):
    # Ler o arquivo Excel
    print("Lendo arquivo Excel...")
    df = pd.read_excel(caminho_arquivo)

    # Criar lista de todas as normas
    todas_normas = []
    print("\nProcessando normas...")

    for idx, normas_string in enumerate(df['Norma'], 1):
        # Pular entradas sem normas
        if isinstance(normas_string, str):  # Verificar se é string
            if normas_string == "não tem norma legal para essa pergunta":
                continue

            # Dividir múltiplas normas da mesma célula
            normas_lista = normas_string.split(';')

            # Limpar cada norma individualmente
            for norma in normas_lista:
                norma_limpa = limpar_norma(norma)
                if norma_limpa:  # Adicionar apenas se não estiver vazia
                    todas_normas.append(norma_limpa)

    # Criar conjunto para remover duplicatas e depois converter para lista ordenada
    normas_unicas = sorted(set(todas_normas))

    # Criar DataFrame com normas únicas
    df_normas_unicas = pd.DataFrame(normas_unicas, columns=['Norma'])

    # Adicionar coluna com numeração
    df_normas_unicas.insert(0, 'ID', range(1, len(df_normas_unicas) + 1))

    # Imprimir estatísticas detalhadas
    print("\nEstatísticas:")
    print(f"Total de normas encontradas (com repetições): {len(todas_normas)}")
    print(f"Total de normas únicas: {len(normas_unicas)}")

    # Calcular e mostrar as normas mais frequentes
    normas_freq = pd.Series(todas_normas).value_counts()
    print("\nTop 10 normas mais frequentes:")
    for norma, freq in normas_freq.head(10).items():
        print(f"{freq} ocorrências: {norma}")

    # Salvar normas únicas em um novo arquivo Excel
    nome_arquivo_saida = 'normas_unicas.xlsx'
    df_normas_unicas.to_excel(nome_arquivo_saida, index=False)

    # Salvar arquivo com frequência das normas
    nome_arquivo_freq = 'frequencia_normas.xlsx'
    df_frequencia = pd.DataFrame({
        'Norma': normas_freq.index,
        'Frequência': normas_freq.values
    })
    df_frequencia.to_excel(nome_arquivo_freq, index=False)

    # Se estiver no Google Colab, fazer o download dos arquivos
    try:
        from google.colab import files
        files.download(nome_arquivo_saida)
        files.download(nome_arquivo_freq)
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados e estão disponíveis para download.")
    except:
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados no diretório atual.")

    # Mostrar as primeiras normas únicas encontradas
    print("\nPrimeiras 10 normas únicas encontradas:")
    print(df_normas_unicas.head(10))

    return df_normas_unicas, df_frequencia

# Usar a função
try:
    caminho_arquivo = 'normas_extraidas.xlsx'  # Arquivo gerado pelo código anterior
    df_normas, df_freq = analisar_normas_excel(caminho_arquivo)
except Exception as e:
    print(f"Erro ao processar o arquivo: {str(e)}")

Lendo arquivo Excel...

Processando normas...

Estatísticas:
Total de normas encontradas (com repetições): 1839
Total de normas únicas: 396

Top 10 normas mais frequentes:
260 ocorrências: RIR/2018
257 ocorrências: Decreto nº 9.580, de 22 de novembro de 2018
122 ocorrências: Instrução Normativa RFB nº 1.500, de 29 de outubro de 2014
74 ocorrências: Lei nº 9.250, de 26 de dezembro de 1995
69 ocorrências: Lei nº 7.713, de 22 de dezembro de 1988
66 ocorrências: Instrução Normativa SRF nº 83, de 11 de outubro de 2001
53 ocorrências: Instrução Normativa SRF nº 208, de 27 de setembro de 2002
48 ocorrências: Instrução Normativa RFB nº 2.178, de 5 de março de 2024
45 ocorrências: Instrução Normativa SRF nº 84, de 11 de outubro de 2001
37 ocorrências: Instrução Normativa RFB nº 1.585, de 31 de agosto de 2015


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Arquivos 'normas_unicas.xlsx' e 'frequencia_normas.xlsx' foram gerados e estão disponíveis para download.

Primeiras 10 normas únicas encontradas:
   ID                                              Norma
0   1                                       AD PGFN nº 3
1   2  Ato Declaratório Cosar nº 47, de 27 de novembr...
2   3  Ato Declaratório Interpretativo RFB nº 1, de 1...
3   4  Ato Declaratório Interpretativo RFB nº 18, de ...
4   5  Ato Declaratório Interpretativo RFB nº 3, de 2...
5   6          Ato Declaratório Interpretativo SRF nº 14
6   7  Ato Declaratório Interpretativo SRF nº 14, de ...
7   8  Ato Declaratório Interpretativo SRF nº 16, de ...
8   9  Ato Declaratório Interpretativo SRF nº 2, de 2...
9  10  Ato Declaratório Interpretativo SRF nº 24, de ...


In [12]:
'''#@title Normalização

import pandas as pd
import re

def padronizar_norma(norma):
    """Função para padronizar o formato da norma removendo datas e outros elementos variáveis"""
    if not isinstance(norma, str):
        return ""

    # Limpar espaços e quebras de linha
    norma = norma.strip().replace('\n', ' ').replace('  ', ' ')

    # Padrões de normalização
    padroes_normalizacao = [
        # Lei (com ou sem data)
        (r'Lei\s+(?:Complementar\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Lei nº {m.group(1)}'),

        # Instrução Normativa
        (r'Instrução\s+Normativa\s+(?:RFB\s+)?(?:SRF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Instrução Normativa RFB nº {m.group(1)}'),

        # Decreto
        (r'Decreto\s*n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Decreto nº {m.group(1)}'),

        # Ato Declaratório RFB
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Ato Declaratório RFB nº {m.group(1)}'),

        # Ato Declaratório SRF
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:SRF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Ato Declaratório SRF nº {m.group(1)}'),

        # Portaria
        (r'Portaria\s+(?:RFB\s+)?(?:MF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Portaria RFB nº {m.group(1)}'),

        # Resolução
        (r'Resolução\s+(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Resolução RFB nº {m.group(1)}'),

        # Medida Provisória
        (r'Medida\s+Provisória\s+n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Medida Provisória nº {m.group(1)}'),

        # Parecer Normativo
        (r'Parecer\s+Normativo\s+(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Parecer Normativo RFB nº {m.group(1)}'),

        # Solução de Consulta
        (r'Solução\s+de\s+Consulta\s+(?:Cosit\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Solução de Consulta Cosit nº {m.group(1)}'),

        # RIR
        (r'RIR/(\d{4})',
         lambda m: f'RIR/{m.group(1)}')
    ]

    # Aplicar cada padrão de normalização
    norma_padronizada = norma
    for padrao, substituicao in padroes_normalizacao:
        match = re.search(padrao, norma, re.IGNORECASE)
        if match:
            norma_padronizada = substituicao(match)
            break

    return norma_padronizada

def analisar_normas_excel(caminho_arquivo):
    # Ler o arquivo Excel
    print("Lendo arquivo Excel...")
    df = pd.read_excel(caminho_arquivo)

    # Criar lista de todas as normas
    todas_normas = []
    print("\nProcessando e padronizando normas...")

    for normas_string in df['Norma']:
        if isinstance(normas_string, str):
            if normas_string == "não tem norma legal para essa pergunta":
                continue

            # Dividir múltiplas normas da mesma célula
            normas_lista = normas_string.split(';')

            # Padronizar cada norma individualmente
            for norma in normas_lista:
                norma_padronizada = padronizar_norma(norma)
                if norma_padronizada:  # Adicionar apenas se não estiver vazia
                    todas_normas.append(norma_padronizada)

    # Criar conjunto para remover duplicatas e depois converter para lista ordenada
    normas_unicas = sorted(set(todas_normas))

    # Criar DataFrame com normas únicas
    df_normas_unicas = pd.DataFrame(normas_unicas, columns=['Norma'])

    # Adicionar coluna com numeração
    df_normas_unicas.insert(0, 'ID', range(1, len(df_normas_unicas) + 1))

    # Imprimir estatísticas detalhadas
    print("\nEstatísticas:")
    print(f"Total de normas encontradas (com repetições): {len(todas_normas)}")
    print(f"Total de normas únicas após padronização: {len(normas_unicas)}")

    # Calcular e mostrar as normas mais frequentes
    normas_freq = pd.Series(todas_normas).value_counts()
    print("\nTop 10 normas mais frequentes:")
    for norma, freq in normas_freq.head(10).items():
        print(f"{freq} ocorrências: {norma}")

    # Salvar normas únicas em um novo arquivo Excel
    nome_arquivo_saida = 'normas_unicas_padronizadas.xlsx'
    df_normas_unicas.to_excel(nome_arquivo_saida, index=False)

    # Salvar arquivo com frequência das normas
    nome_arquivo_freq = 'frequencia_normas_padronizadas.xlsx'
    df_frequencia = pd.DataFrame({
        'Norma': normas_freq.index,
        'Frequência': normas_freq.values
    })
    df_frequencia.to_excel(nome_arquivo_freq, index=False)

    # Se estiver no Google Colab, fazer o download dos arquivos
    try:
        from google.colab import files
        files.download(nome_arquivo_saida)
        files.download(nome_arquivo_freq)
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados e estão disponíveis para download.")
    except:
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados no diretório atual.")

    # Mostrar as primeiras normas únicas encontradas
    print("\nPrimeiras 10 normas únicas encontradas:")
    print(df_normas_unicas.head(10))

    return df_normas_unicas, df_frequencia

# Usar a função
try:
    caminho_arquivo = 'normas_extraidas.xlsx'  # Arquivo gerado pelo código anterior
    df_normas, df_freq = analisar_normas_excel(caminho_arquivo)
except Exception as e:
    print(f"Erro ao processar o arquivo: {str(e)}")'''

Lendo arquivo Excel...

Processando e padronizando normas...

Estatísticas:
Total de normas encontradas (com repetições): 1790
Total de normas únicas após padronização: 301

Top 10 normas mais frequentes:
260 ocorrências: RIR/2018
260 ocorrências: Decreto nº 9.580
123 ocorrências: Instrução Normativa RFB nº 1.500
79 ocorrências: Lei nº 7.713
77 ocorrências: Lei nº 9.250
67 ocorrências: Instrução Normativa RFB nº 83
56 ocorrências: Instrução Normativa RFB nº 208
48 ocorrências: Instrução Normativa RFB nº 2.178
47 ocorrências: Instrução Normativa RFB nº 84
39 ocorrências: Instrução Normativa RFB nº 1.585


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Arquivos 'normas_unicas_padronizadas.xlsx' e 'frequencia_normas_padronizadas.xlsx' foram gerados e estão disponíveis para download.

Primeiras 10 normas únicas encontradas:
   ID                       Norma
0   1   Ato Declaratório RFB nº 1
1   2  Ato Declaratório RFB nº 16
2   3  Ato Declaratório RFB nº 18
3   4   Ato Declaratório RFB nº 3
4   5           Decreto nº 21.177
5   6           Decreto nº 27.784
6   7            Decreto nº 3.000
7   8              Decreto nº 361
8   9            Decreto nº 4.897
9  10            Decreto nº 5.128


In [18]:
#@title Normalização

import pandas as pd
import re

def padronizar_norma(norma):
    """Função para padronizar o formato da norma removendo datas e outros elementos variáveis"""
    if not isinstance(norma, str):
        return ""

    # Limpar espaços e quebras de linha
    norma = norma.strip().replace('\n', ' ').replace('  ', ' ')

    # Padrões de normalização
    padroes_normalizacao = [
        # Padrões PGFN
        (r'Parecer\s+SEI\s+(?:N[º°]\s+)?([\d\.]+/\d{4}/(?:CRJ/PGACET/)?PGFN(?:-MF|-ME)?)',
         lambda m: f'Parecer SEI nº {m.group(1)}'),
        (r'Nota\s+SEI\s+(?:N[º°]\s+)?([\d\.]+/\d{4}/(?:CRJ/PGACET/)?PGFN(?:-MF|-ME)?)',
         lambda m: f'Nota SEI nº {m.group(1)}'),
        (r'Parecer\s+PGFN/(?:CAT|PGA|CRJ)/(?:N[º°]\s+)?([\d\.]+(?:/\d{4})?)',
         lambda m: f'Parecer PGFN nº {m.group(1)}'),
        (r'Nota\s+PGFN\s+(?:CRJ\s+)?(?:N[º°]\s+)?([\d\.]+/\d{4})',
         lambda m: f'Nota PGFN nº {m.group(1)}'),
        (r'(?:Ato\s+Declaratório\s+)?PGFN(?:/CRJ)?(?:/N[º°]\s+)?([\d\.]+(?:/\d{4})?)',
         lambda m: f'Ato Declaratório PGFN nº {m.group(1)}'),
        (r'AD\s+PGFN\s+(?:N[º°]\s+)?([\d\.]+)',
         lambda m: f'Ato Declaratório PGFN nº {m.group(1)}'),
        (r'Ato\s+Declaratório\s+PGFN\s+(?:N[º°]\s+)?([\d\.]+)',
         lambda m: f'Ato Declaratório PGFN nº {m.group(1)}'),

        # Instrução Normativa
        (r'Instrução\s+Normativa\s+(?:RFB\s+)?(?:SRF\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Instrução Normativa nº {m.group(1)}'),

        # Lei
        (r'Lei\s+(?:Complementar\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Lei nº {m.group(1)}'),

        # Decreto
        (r'Decreto\s+N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Decreto nº {m.group(1)}'),

        # Ato Declaratório RFB
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:RFB\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório RFB nº {m.group(1)}'),

        # Ato Declaratório SRF
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:SRF\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório SRF nº {m.group(1)}'),

        # Ato Declaratório CST
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:CST\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório CST nº {m.group(1)}'),

        # Ato Declaratório COSIT
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:Cosit\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório Cosit nº {m.group(1)}'),

        # Ato Declaratório COSAR
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:Cosar\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório Cosar nº {m.group(1)}'),

        # Portaria
        (r'Portaria\s+(?:RFB\s+)?(?:MF\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Portaria nº {m.group(1)}'),

        # Resolução
        (r'Resolução\s+(?:RFB\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Resolução nº {m.group(1)}'),

        # Medida Provisória
        (r'Medida\s+Provisória\s+N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Medida Provisória nº {m.group(1)}'),

        # Parecer Normativo
        (r'Parecer\s+Normativo\s+(?:RFB\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Parecer Normativo nº {m.group(1)}'),

        # Solução de Consulta
        (r'Solução\s+de\s+Consulta\s+(?:Cosit\s+)?N[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Solução de Consulta nº {m.group(1)}'),

        # RIR
        (r'RIR/(\d{4})',
         lambda m: f'RIR/{m.group(1)}')
    ]

    # Aplicar cada padrão de normalização
    norma_padronizada = norma
    for padrao, substituicao in padroes_normalizacao:
        match = re.search(padrao, norma, re.IGNORECASE)
        if match:
            norma_padronizada = substituicao(match)
            break  # Sai após o primeiro padrão correspondido

    return norma_padronizada

def analisar_normas_excel(caminho_arquivo):
    # Ler o arquivo Excel
    print("Lendo arquivo Excel...")
    df = pd.read_excel(caminho_arquivo)

    # Criar lista de todas as normas
    todas_normas = []
    print("\nProcessando e padronizando normas...")

    for normas_string in df['Norma']:
        if isinstance(normas_string, str):
            if normas_string.lower() == "não tem norma legal para essa pergunta":
                continue

            # Dividir múltiplas normas da mesma célula
            normas_lista = re.split(r';|\n', normas_string)

            # Padronizar cada norma individualmente
            for norma in normas_lista:
                norma_padronizada = padronizar_norma(norma)
                if norma_padronizada:  # Adicionar apenas se não estiver vazia
                    todas_normas.append(norma_padronizada)

    # Criar conjunto para remover duplicatas e depois converter para lista ordenada
    normas_unicas = sorted(set(todas_normas))

    # Criar DataFrame com normas únicas
    df_normas_unicas = pd.DataFrame(normas_unicas, columns=['Norma'])

    # Adicionar coluna com numeração
    df_normas_unicas.insert(0, 'ID', range(1, len(df_normas_unicas) + 1))

    # Imprimir estatísticas detalhadas
    print("\nEstatísticas:")
    print(f"Total de normas encontradas (com repetições): {len(todas_normas)}")
    print(f"Total de normas únicas após padronização: {len(normas_unicas)}")

    # Calcular e mostrar as normas mais frequentes
    normas_freq = pd.Series(todas_normas).value_counts()
    print("\nTop 10 normas mais frequentes:")
    for norma, freq in normas_freq.head(10).items():
        print(f"{freq} ocorrências: {norma}")

    # Salvar normas únicas em um novo arquivo Excel
    nome_arquivo_saida = 'normas_unicas_padronizadas.xlsx'
    df_normas_unicas.to_excel(nome_arquivo_saida, index=False)

    # Salvar arquivo com frequência das normas
    nome_arquivo_freq = 'frequencia_normas_padronizadas.xlsx'
    df_frequencia = pd.DataFrame({
        'Norma': normas_freq.index,
        'Frequência': normas_freq.values
    })
    df_frequencia.to_excel(nome_arquivo_freq, index=False)

    # Mostrar as primeiras normas únicas encontradas
    print("\nPrimeiras 10 normas únicas encontradas:")
    print(df_normas_unicas.head(10))

    return df_normas_unicas, df_frequencia

# Usar a função
try:
    caminho_arquivo = 'normas_extraidas.xlsx'  # Arquivo gerado pelo código anterior
    df_normas, df_freq = analisar_normas_excel(caminho_arquivo)
except Exception as e:
    print(f"Erro ao processar o arquivo: {str(e)}")


Lendo arquivo Excel...

Processando e padronizando normas...

Estatísticas:
Total de normas encontradas (com repetições): 1839
Total de normas únicas após padronização: 336

Top 10 normas mais frequentes:
260 ocorrências: RIR/2018
260 ocorrências: Decreto nº 9.580
123 ocorrências: Instrução Normativa nº 1.500
79 ocorrências: Lei nº 7.713
77 ocorrências: Lei nº 9.250
67 ocorrências: Instrução Normativa nº 83
56 ocorrências: Instrução Normativa nº 208
48 ocorrências: Instrução Normativa nº 2.178
47 ocorrências: Instrução Normativa nº 84
39 ocorrências: Instrução Normativa nº 1.585

Primeiras 10 normas únicas encontradas:
   ID                         Norma
0   1  Ato Declaratório Cosar nº 47
1   2    Ato Declaratório PGFN nº .
2   3    Ato Declaratório PGFN nº 1
3   4   Ato Declaratório PGFN nº 13
4   5   Ato Declaratório PGFN nº 14
5   6    Ato Declaratório PGFN nº 2
6   7    Ato Declaratório PGFN nº 3
7   8    Ato Declaratório PGFN nº 4
8   9    Ato Declaratório PGFN nº 5
9  10    Ato 

In [13]:
'''#@title Padronização

import pandas as pd
import re

def padronizar_norma(norma):
    """Função para padronizar o formato da norma removendo datas e outros elementos variáveis"""
    if not isinstance(norma, str):
        return ""

    # Limpar espaços e quebras de linha
    norma = norma.strip().replace('\n', ' ').replace('  ', ' ')

    # Verificar primeiro se é o Decreto 9.580 e converter para RIR/2018
    if re.search(r'Decreto\s*n[º°]\s*9\.580(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?', norma, re.IGNORECASE):
        return 'RIR/2018'

    # Padrões de normalização
    padroes_normalizacao = [
        # Lei (com ou sem data)
        (r'Lei\s+(?:Complementar\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Lei nº {m.group(1)}'),

        # Instrução Normativa
        (r'Instrução\s+Normativa\s+(?:RFB\s+)?(?:SRF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Instrução Normativa RFB nº {m.group(1)}'),

        # Decreto
        (r'Decreto\s*n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Decreto nº {m.group(1)}'),

        # Ato Declaratório RFB
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Ato Declaratório RFB nº {m.group(1)}'),

        # Ato Declaratório SRF
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:SRF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Ato Declaratório SRF nº {m.group(1)}'),

        # Portaria
        (r'Portaria\s+(?:RFB\s+)?(?:MF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Portaria RFB nº {m.group(1)}'),

        # Resolução
        (r'Resolução\s+(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Resolução RFB nº {m.group(1)}'),

        # Medida Provisória
        (r'Medida\s+Provisória\s+n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Medida Provisória nº {m.group(1)}'),

        # Parecer Normativo
        (r'Parecer\s+Normativo\s+(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Parecer Normativo RFB nº {m.group(1)}'),

        # Solução de Consulta
        (r'Solução\s+de\s+Consulta\s+(?:Cosit\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de\s*[\d\.]+\s+de\s+\w+\s+de\s+\d{4})?',
         lambda m: f'Solução de Consulta Cosit nº {m.group(1)}'),

        # RIR
        (r'RIR/(\d{4})',
         lambda m: f'RIR/{m.group(1)}')
    ]

    # Aplicar cada padrão de normalização
    norma_padronizada = norma
    for padrao, substituicao in padroes_normalizacao:
        match = re.search(padrao, norma, re.IGNORECASE)
        if match:
            norma_padronizada = substituicao(match)
            break

    return norma_padronizada

def analisar_normas_excel(caminho_arquivo):
    # Ler o arquivo Excel
    print("Lendo arquivo Excel...")
    df = pd.read_excel(caminho_arquivo)

    # Criar lista de todas as normas
    todas_normas = []
    print("\nProcessando e padronizando normas...")

    for normas_string in df['Norma']:
        if isinstance(normas_string, str):
            if normas_string == "não tem norma legal para essa pergunta":
                continue

            # Dividir múltiplas normas da mesma célula
            normas_lista = normas_string.split(';')

            # Padronizar cada norma individualmente
            for norma in normas_lista:
                norma_padronizada = padronizar_norma(norma)
                if norma_padronizada:  # Adicionar apenas se não estiver vazia
                    todas_normas.append(norma_padronizada)

    # Criar conjunto para remover duplicatas e depois converter para lista ordenada
    normas_unicas = sorted(set(todas_normas))

    # Criar DataFrame com normas únicas
    df_normas_unicas = pd.DataFrame(normas_unicas, columns=['Norma'])

    # Adicionar coluna com numeração
    df_normas_unicas.insert(0, 'ID', range(1, len(df_normas_unicas) + 1))

    # Imprimir estatísticas detalhadas
    print("\nEstatísticas:")
    print(f"Total de normas encontradas (com repetições): {len(todas_normas)}")
    print(f"Total de normas únicas após padronização: {len(normas_unicas)}")

    # Calcular e mostrar as normas mais frequentes
    normas_freq = pd.Series(todas_normas).value_counts()
    print("\nTop 10 normas mais frequentes:")
    for norma, freq in normas_freq.head(10).items():
        print(f"{freq} ocorrências: {norma}")

    # Salvar normas únicas em um novo arquivo Excel
    nome_arquivo_saida = 'normas_unicas_padronizadas.xlsx'
    df_normas_unicas.to_excel(nome_arquivo_saida, index=False)

    # Salvar arquivo com frequência das normas
    nome_arquivo_freq = 'frequencia_normas_padronizadas.xlsx'
    df_frequencia = pd.DataFrame({
        'Norma': normas_freq.index,
        'Frequência': normas_freq.values
    })
    df_frequencia.to_excel(nome_arquivo_freq, index=False)

    # Se estiver no Google Colab, fazer o download dos arquivos
    try:
        from google.colab import files
        files.download(nome_arquivo_saida)
        files.download(nome_arquivo_freq)
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados e estão disponíveis para download.")
    except:
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados no diretório atual.")

    # Mostrar as primeiras normas únicas encontradas
    print("\nPrimeiras 10 normas únicas encontradas:")
    print(df_normas_unicas.head(10))

    return df_normas_unicas, df_frequencia

# Usar a função
try:
    caminho_arquivo = 'normas_extraidas.xlsx'  # Arquivo gerado pelo código anterior
    df_normas, df_freq = analisar_normas_excel(caminho_arquivo)
except Exception as e:
    print(f"Erro ao processar o arquivo: {str(e)}")'''

Lendo arquivo Excel...

Processando e padronizando normas...

Estatísticas:
Total de normas encontradas (com repetições): 1790
Total de normas únicas após padronização: 300

Top 10 normas mais frequentes:
520 ocorrências: RIR/2018
123 ocorrências: Instrução Normativa RFB nº 1.500
79 ocorrências: Lei nº 7.713
77 ocorrências: Lei nº 9.250
67 ocorrências: Instrução Normativa RFB nº 83
56 ocorrências: Instrução Normativa RFB nº 208
48 ocorrências: Instrução Normativa RFB nº 2.178
47 ocorrências: Instrução Normativa RFB nº 84
39 ocorrências: Instrução Normativa RFB nº 1.585
24 ocorrências: Lei nº 10.406


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Arquivos 'normas_unicas_padronizadas.xlsx' e 'frequencia_normas_padronizadas.xlsx' foram gerados e estão disponíveis para download.

Primeiras 10 normas únicas encontradas:
   ID                       Norma
0   1   Ato Declaratório RFB nº 1
1   2  Ato Declaratório RFB nº 16
2   3  Ato Declaratório RFB nº 18
3   4   Ato Declaratório RFB nº 3
4   5           Decreto nº 21.177
5   6           Decreto nº 27.784
6   7            Decreto nº 3.000
7   8              Decreto nº 361
8   9            Decreto nº 4.897
9  10            Decreto nº 5.128


In [19]:
#@title Padronização

import pandas as pd
import re

def padronizar_norma(norma):
    """Função para padronizar o formato da norma removendo datas e outros elementos variáveis"""
    if not isinstance(norma, str):
        return ""

    # Limpar espaços e quebras de linha
    norma = norma.strip().replace('\n', ' ').replace('  ', ' ')

    # Verificar primeiro se é o Decreto nº 9.580 e converter para RIR/2018
    if re.search(r'Decreto\s*n[º°]\s*9\.580(?:\s*,\s*de.*)?', norma, re.IGNORECASE):
        return 'RIR/2018'

    # Padrões de normalização
    padroes_normalizacao = [
        # Padrões PGFN
        (r'Parecer\s+SEI\s+(?:N[º°]\s+)?([\d\.]+/\d{4}/(?:CRJ/PGACET/)?PGFN(?:-MF|-ME)?)',
         lambda m: f'Parecer SEI nº {m.group(1)}'),
        (r'Nota\s+SEI\s+(?:N[º°]\s+)?([\d\.]+/\d{4}/(?:CRJ/PGACET/)?PGFN(?:-MF|-ME)?)',
         lambda m: f'Nota SEI nº {m.group(1)}'),
        (r'Parecer\s+PGFN/(?:CAT|PGA|CRJ)/(?:N[º°]\s+)?([\d\.]+(?:/\d{4})?)',
         lambda m: f'Parecer PGFN nº {m.group(1)}'),
        (r'Nota\s+PGFN\s+(?:CRJ\s+)?(?:N[º°]\s+)?([\d\.]+/\d{4})',
         lambda m: f'Nota PGFN nº {m.group(1)}'),
        (r'(?:Ato\s+Declaratório\s+)?PGFN(?:/CRJ)?(?:/N[º°]\s+)?([\d\.]+(?:/\d{4})?)',
         lambda m: f'Ato Declaratório PGFN nº {m.group(1)}'),
        (r'AD\s+PGFN\s+(?:N[º°]\s+)?([\d\.]+)',
         lambda m: f'Ato Declaratório PGFN nº {m.group(1)}'),
        (r'Ato\s+Declaratório\s+PGFN\s+(?:N[º°]\s+)?([\d\.]+)',
         lambda m: f'Ato Declaratório PGFN nº {m.group(1)}'),

        # Lei (com ou sem data)
        (r'Lei\s+(?:Complementar\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Lei nº {m.group(1)}'),

        # Instrução Normativa
        (r'Instrução\s+Normativa\s+(?:RFB\s+)?(?:SRF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Instrução Normativa RFB nº {m.group(1)}'),

        # Decreto (geral)
        (r'Decreto\s*n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Decreto nº {m.group(1)}'),

        # Ato Declaratório RFB
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório RFB nº {m.group(1)}'),

        # Ato Declaratório SRF
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:SRF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório SRF nº {m.group(1)}'),

        # Ato Declaratório CST
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:CST\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório CST nº {m.group(1)}'),

        # Ato Declaratório Cosit
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:Cosit\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório Cosit nº {m.group(1)}'),

        # Ato Declaratório Cosar
        (r'Ato\s+Declaratório\s+(?:Interpretativo\s+)?(?:Cosar\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Ato Declaratório Cosar nº {m.group(1)}'),

        # Portaria
        (r'Portaria\s+(?:RFB\s+)?(?:MF\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Portaria nº {m.group(1)}'),

        # Resolução
        (r'Resolução\s+(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Resolução nº {m.group(1)}'),

        # Medida Provisória
        (r'Medida\s+Provisória\s+n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Medida Provisória nº {m.group(1)}'),

        # Parecer Normativo
        (r'Parecer\s+Normativo\s+(?:RFB\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Parecer Normativo RFB nº {m.group(1)}'),

        # Solução de Consulta
        (r'Solução\s+de\s+Consulta\s+(?:Cosit\s+)?n[º°]\s*([\d\.]+)(?:\s*,\s*de.*)?',
         lambda m: f'Solução de Consulta Cosit nº {m.group(1)}'),

        # RIR
        (r'RIR/(\d{4})',
         lambda m: f'RIR/{m.group(1)}')
    ]

    # Aplicar cada padrão de normalização
    norma_padronizada = norma
    for padrao, substituicao in padroes_normalizacao:
        match = re.search(padrao, norma, re.IGNORECASE)
        if match:
            norma_padronizada = substituicao(match)
            break

    return norma_padronizada

def analisar_normas_excel(caminho_arquivo):
    # Ler o arquivo Excel
    print("Lendo arquivo Excel...")
    df = pd.read_excel(caminho_arquivo)

    # Criar lista de todas as normas
    todas_normas = []
    print("\nProcessando e padronizando normas...")

    for normas_string in df['Norma']:
        if isinstance(normas_string, str):
            if normas_string.lower() == "não tem norma legal para essa pergunta":
                continue

            # Dividir múltiplas normas da mesma célula
            normas_lista = re.split(r';|\n', normas_string)

            # Padronizar cada norma individualmente
            for norma in normas_lista:
                norma_padronizada = padronizar_norma(norma)
                if norma_padronizada:  # Adicionar apenas se não estiver vazia
                    todas_normas.append(norma_padronizada)

    # Criar conjunto para remover duplicatas e depois converter para lista ordenada
    normas_unicas = sorted(set(todas_normas))

    # Criar DataFrame com normas únicas
    df_normas_unicas = pd.DataFrame(normas_unicas, columns=['Norma'])

    # Adicionar coluna com numeração
    df_normas_unicas.insert(0, 'ID', range(1, len(df_normas_unicas) + 1))

    # Imprimir estatísticas detalhadas
    print("\nEstatísticas:")
    print(f"Total de normas encontradas (com repetições): {len(todas_normas)}")
    print(f"Total de normas únicas após padronização: {len(normas_unicas)}")

    # Calcular e mostrar as normas mais frequentes
    normas_freq = pd.Series(todas_normas).value_counts()
    print("\nTop 10 normas mais frequentes:")
    for norma, freq in normas_freq.head(10).items():
        print(f"{freq} ocorrências: {norma}")

    # Salvar normas únicas em um novo arquivo Excel
    nome_arquivo_saida = 'normas_unicas_padronizadas.xlsx'
    df_normas_unicas.to_excel(nome_arquivo_saida, index=False)

    # Salvar arquivo com frequência das normas
    nome_arquivo_freq = 'frequencia_normas_padronizadas.xlsx'
    df_frequencia = pd.DataFrame({
        'Norma': normas_freq.index,
        'Frequência': normas_freq.values
    })
    df_frequencia.to_excel(nome_arquivo_freq, index=False)

    # Se estiver no Google Colab, fazer o download dos arquivos
    try:
        from google.colab import files
        files.download(nome_arquivo_saida)
        files.download(nome_arquivo_freq)
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados e estão disponíveis para download.")
    except:
        print(f"\nArquivos '{nome_arquivo_saida}' e '{nome_arquivo_freq}' foram gerados no diretório atual.")

    # Mostrar as primeiras normas únicas encontradas
    print("\nPrimeiras 10 normas únicas encontradas:")
    print(df_normas_unicas.head(10))

    return df_normas_unicas, df_frequencia

# Usar a função
try:
    caminho_arquivo = 'normas_extraidas.xlsx'  # Arquivo gerado pelo código anterior
    df_normas, df_freq = analisar_normas_excel(caminho_arquivo)
except Exception as e:
    print(f"Erro ao processar o arquivo: {str(e)}")


Lendo arquivo Excel...

Processando e padronizando normas...

Estatísticas:
Total de normas encontradas (com repetições): 1839
Total de normas únicas após padronização: 335

Top 10 normas mais frequentes:
520 ocorrências: RIR/2018
123 ocorrências: Instrução Normativa RFB nº 1.500
79 ocorrências: Lei nº 7.713
77 ocorrências: Lei nº 9.250
67 ocorrências: Instrução Normativa RFB nº 83
56 ocorrências: Instrução Normativa RFB nº 208
48 ocorrências: Instrução Normativa RFB nº 2.178
47 ocorrências: Instrução Normativa RFB nº 84
39 ocorrências: Instrução Normativa RFB nº 1.585
24 ocorrências: Lei nº 10.406


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Arquivos 'normas_unicas_padronizadas.xlsx' e 'frequencia_normas_padronizadas.xlsx' foram gerados e estão disponíveis para download.

Primeiras 10 normas únicas encontradas:
   ID                         Norma
0   1  Ato Declaratório Cosar nº 47
1   2    Ato Declaratório PGFN nº .
2   3    Ato Declaratório PGFN nº 1
3   4   Ato Declaratório PGFN nº 13
4   5   Ato Declaratório PGFN nº 14
5   6    Ato Declaratório PGFN nº 2
6   7    Ato Declaratório PGFN nº 3
7   8    Ato Declaratório PGFN nº 4
8   9    Ato Declaratório PGFN nº 5
9  10    Ato Declaratório PGFN nº 6
